In [56]:
# 이 코드에서는 깨끗한 단어를 추출하기 위해 심화단계의 tokenize를 수행해 봅니다. (NLP를 위한 전처리)
# 그동안 split()만 사용해 왔지만 다른 방법도 적용해 봅니다. 어느 feature가 좋은지 모르므로 최대한 많이 뽑아내는 것이 목적입니다. 

from konlpy.tag import Kkma # 형태소 분석기 중에는 kkma가 성능이 제일 좋음.
ma = Kkma() # 형태소 분석기 인스턴스
sentence = "오늘 미세먼지는 어제 미세먼지보다 나빠요."

# 명사를 뽑으라고 시키면 => 오늘, 미세, 먼지, 어제, 미세, 먼지가 뽑혀야 함.
# ma.pos(sentence) # 형태소가 부착된(태깅된) 형태로 반환
print([token[0] for token in ma.pos(sentence) if token[1].startswith("NN")])
print(ma.nouns(sentence)) # 명사만 뽑기

# BOW => index term, Lexicon(dictionary)로 부르기도 함. 
# 문장 단위 -> 어절 단위 -> 형태소 단위 -> 품사(명사) 단위로 보고, 추가로 Ngram 단위로도 문장을 볼 예정
# 일반적인 전처리 순서 : 토크나이징 -> Normalization(단어의 길이, 한국어의 경우 1음절 단위로 수행하기도 함)
from nltk.tokenize import sent_tokenize, word_tokenize # 두 개의 토큰화 모듈 임포트 

# 구두점에 대한 처리만 빼고 비슷한 결과가 나올 것임.
print(sentence.split()) # 단순히 split
print(word_tokenize(sentence)) # 구두점도 별도로 분류를 했으므로 어절이 누구인지 찾을 수 있다.

print("원본 : ", sentence)

# tokenized data
lexicon = list()
th = 1 # 1음절보다 큰 문자를 고르기 위한 상수
lexicon = [token for token in word_tokenize(sentence) if len(token) > th]
print("어절 단위 분리 : ", lexicon)

# 품사에 대해 태깅 작업 수행
# print(ma.pos(sentence)) # 둘의 차이는 그렇게 크지 않다.
for token in [token for token in word_tokenize(sentence) if len(token) > th]: # 명사 분류시 문제가 될 수 있으므로 토큰화한 후 수행
    lexicon.extend([token[0] for token in ma.pos(token) if len(token[0]) > th]) # (단어, 품사) 튜플쌍으로 받기 위해 for문을 돌면서 형태소 분석. (ma.morphs() 함수를 사용하면 형태소만 반환받을 수 있음. )
    # lexicon.extend([token[0] for token in ma.pos(token) if len(token[0]) > th] and token[1] in ["NN", "NNG"]) 등과 같이 and조건 옆에 원하는 품사를 넣어서 그 POS 태그에 해당하는 것들만 튜플쌍으로 받을 수도 있다. 
print("형태소 분석 결과 : ", list(set(lexicon))) # 필요없는 중복값을 날리기 위해 set()에 담음 

for token in [token for token in word_tokenize(sentence) if len(token) > th]: # 명사 분류시 문제가 될 수 있으므로 토큰화한 후 수행
    lexicon.extend(ma.nouns(token))
print("명사 분석 결과 : ", list(set(lexicon))) # 필요없는 중복값을 날리기 위해 set()에 담음 
# 더 세밀한 분석을 위해 N-gram을 적용할 필요가 있다.

lexicon.extend(ngramEojeol(" ".join([token[0] for token in ma.pos(sentence)])))
print("어절(바이그램) 분석 : ", list(set(lexicon)))

newLexicon = list()
for term in lexicon:
    newLexicon.extend(ngramUmjeol(term))
lexicon.extend([term for term in newLexicon if len(term.strip()) > th])
print("음절(바이그램) 분석 : ", list(set(lexicon)))
print(len(list(set(lexicon))))


['오늘', '미세', '먼지', '미세', '먼지']
['오늘', '미세', '미세먼지', '먼지']
['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요.']
['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요', '.']
원본 :  오늘 미세먼지는 어제 미세먼지보다 나빠요.
어절 단위 분리 :  ['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요']
형태소 분석 결과 :  ['미세먼지는', '아요', '보다', '어제', '먼지', '나쁘', '미세먼지보다', '미세', '오늘', '나빠요']
명사 분석 결과 :  ['미세먼지는', '아요', '보다', '어제', '미세먼지', '먼지', '나쁘', '미세먼지보다', '미세', '오늘', '나빠요']
어절(바이그램) 분석 :  ['미세', '나빠요', '미세먼지는', '먼지 보다', '는 어제', '보다', '미세 먼지', '먼지 는', '오늘', '오늘 미세', '어제', '미세먼지', '먼지', '보다 나쁘', '미세먼지보다', '아요', '아요 .', '나쁘 아요', '나쁘', '어제 미세']
음절(바이그램) 분석 :  ['나빠', '지보', '미세', '나빠요', '세먼', '미세먼지는', '먼지 보다', '는 어제', '보다', '미세 먼지', '지는', '먼지 는', '오늘', '오늘 미세', '어제', '미세먼지', '먼지', '보다 나쁘', '미세먼지보다', '아요', '아요 .', '나쁘 아요', '빠요', '나쁘', '어제 미세']
25


In [57]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [58]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [59]:
from os import listdir

# path = "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/" (상대경로)
# path = "0314_DownloadedNewstxts" (절대경로)

# fileids() => 말뭉치 목록을 리턴. 이 함수처럼 getFileList라는 함수를 만들자.  
def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

In [60]:
def getContent(file): # txt 컨텐츠를 편하게 읽어오기 위한 함수
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

In [82]:
len(getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")) # 정상작동하는지 확인

180

In [83]:
content = getContent(getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")[0]) # 정상작동하는지 확인

~~~
'\n\n\n\n\n// flash \nfunction _flash_removeCallback() {}\n\n jawon1212@donga.com]\n\n'와 같이 거슬리는 단어를 정규식을 응용해 걸러낼 필요가 있다. 
~~~

In [103]:
import re
from string import punctuation # !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~와 같은 불용어가 들어 있다. 
# 문자를 기입할때는 보통 []로 감싸서 사용. 

# 패턴의 리스트를 돌려주는 함수(key로 pattern을 호출해서 re.sub을 적용하기 위한 목적)
def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{2,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    
    return patternList
    
    
# for _ in ["Email", "Domain", "Nonword", "Punctuation", "Whitespace"]:
    
    
# # 첫번째 필터링 : 특수문자 제거
# pattern = re.compile(r"[%s]{2,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의 
# print("------------------------------------------------------------------------------------------------------------------")
# print("1st pattern : ",  pattern) # 패턴이 잘 컴파일되었는지 확인 목적  
# print("1st findall result : ", pattern.findall(content)) # 패턴대로 잘 찾아지는지 확인 목적
# print("1st filtered result : ", pattern.sub(" ", content)) # 정의된 패턴을 통해 2번이상 반복되는 특수 문자를 제거한 결과물(" "으로 치환한 결과) 출력
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 두번째 필터링 : 이메일 주소 제거
# pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") # 이메일은 한글을 사용하지 못하므로 영어, 숫자, 하이픈, . 등만 존재할 것이다. 그 다음 @이 나옴.
# print("2nd pattern : ",  pattern2)
# print("2nd findall result : ", pattern2.search(content)) # 매치되는 딱 하나의 결과만 리턴
# print("2nd filtered result : ", pattern2.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 세번째 필터링 : 언론사 도메인 주소 제거 (ex. www.khan.co.kr)
# pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # www.domain.com일 경우, www과 같은 서브도메인이 붙고 후속 내용들이 뒤따르는 형태. 
# print("3rd pattern : ",  pattern3)
# print("3rd findall result : ", pattern3.findall(content)) 
# print("3rd filtered result : ", pattern3.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 네번째 필터링 : 화이트스페이스 제거
# pattern4 = re.compile(r"\s{2,}") # 공백이 두번이상 반복되는 케이스에 대한 패턴 정의
# print("4th pattern : ",  pattern4)
# print("4th findall result : ", pattern4.findall(content)) 
# print("4th filtered result : ", pattern4.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 다섯번째 필터링 : 한글이 아닌 영어 및 기호 제거
# pattern5 = re.compile(r"([^ㄱ_ㅎㅏ_ㅣ가_힣0-9]+)")
# print("5th pattern : ",  pattern5)
# print("5th findall result : ", pattern5.findall(content)) 
# print("5th filtered result : ", pattern5.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")

# _flash_remoceCallback같은 거슬리는 영어 문구 제거(보통 EMFscientiest와 같이 특별한 의미를 갖고 있는 영단어 때문에 사용하면 위험함)
# pattern = re.compile(r"[A-Za-z\-\_]{4,}") # 4글자 이상 반복되는 영어 문구 찾아내기
# print("pattern : ",  pattern)
# print("findall result : ", pattern.findall(content))
# print("filtered result : ", pattern.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")

In [104]:
patternList = getPatternList()

content = getContent(getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")[0])
for _ in ["Korean", "Whitespace"]:
    content = patternList[_].sub(" ", content)
print(content)

 오류를 우회하기 위한 함수 추가 오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원 기자 1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과 중앙일보 등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과 유엔환경계획 에 국제 과학자 호소문 을 제출하기는 했다

In [111]:
from nltk.tokenize import sent_tokenize

len(sent_tokenize(content)) # 구두점이 사라져서 토큰화 못함.

content = getContent(getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")[0])
for _ in ["Email", "Domain", "Korean", "Punctuation", "Whitespace"]:
    content = patternList[_].sub(" ", content)
    
termList = list()
posList = list()
nounList = list()
ngramList = list()

for sentence in sent_tokenize(content):
    for token in word_tokenize(sentence):
        if len(token) > th:
            termList.append(token)
            # 아래서부터는 list이기때문에 extend를 사용해야 함. 
            posList.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
            nounList.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
            ngramList.extend(ngramUmjeol(token)) # 바이그램을 리턴
        
# 빠른 속도를 위해 set사용
termList = list(set(termList))
posList = list(set(posList))
nounList = list(set(nounList))
ngramList = list(set(ngramList))

In [110]:
len(termList), len(posList), len(nounList), len(ngramList) # (282, 219, 188, 492)

# len(termList) : 순수하게 어절 단위로 자른 차원 수를 알 수 있고,(282차원)
# th(길이)가 1보다 큰것들로 분류했으므로 은, 는, 이, 가와 같은 문자들이 사라졌을것. 
# len(posList), len(nounList)는 큰 차이가 없음. 즉 한국어는 명사 위주로 중요한 내용을 표현함을 알 수 있다. 
# ngram 모델의 단점 중 space complexity가 높다는 문제를 len(ngramList)을 통해 알 수 있다. 

(282, 219, 188, 492)

In [115]:
# 문서가 크면 클수록 계속해서 선형적으로 늘어난다. 이런식으로 뽑아낸 대량의 lexicon을 두고 query에 대해 잘 retrieval하는 것이 목적. 
len(list(set(termList + posList + nounList + ngramList))) # 최종적으로 뽑은 term set (lexicon, index term list라고 부름.)

764